In [1]:
import os

import numpy as np
import pandas as pd
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

In [2]:
def multi_log_loss(y_true, y_pred):
    assert y_true.shape == y_pred.shape
    columns = y_true.shape[1]
    column_losses = []
    for i in range(0, columns):
        column_losses.append(log_loss(y_true[:, i], y_pred[:, i]))
    return np.array(column_losses).mean()

In [15]:
LABEL_COLUMNS = ['toxic', 'severe_toxic','obscene','threat','insult','identity_hate']

SINGLE_DIR = '/mnt/ml-team/minerva/toxic/single_model_predictions'

BWCL_VALID_PATH = os.path.join(SINGLE_DIR, 'valid', 'logreg_bad_word_count.csv') # 0.0669
CVDCNN_VALID_PATH = os.path.join(SINGLE_DIR, 'valid', 'char_vdcnn.csv') # 0.0435
CLOGREG_VALID_PATH = os.path.join(SINGLE_DIR, 'valid', 'logreg_count.csv') # 0.126
GDPCNN_VALID_PATH = os.path.join(SINGLE_DIR, 'valid', 'glove_dpcnn.csv') # 0.0422
GLSTM_VALID_PATH = os.path.join(SINGLE_DIR, 'valid', 'glove_lstm.csv') # 0.0417
GSCNN_VALID_PATH = os.path.join(SINGLE_DIR, 'valid', 'glove_scnn.csv') # 0.0427
TFIDIF_LOGREG_VALID_PATH = os.path.join(SINGLE_DIR, 'valid', 'logreg_tfidf.csv') # 0.0459
WLSTM_VALID_PATH = os.path.join(SINGLE_DIR, 'valid', 'word_lstm.csv') # 0.0486

VALID_LABELS_PATH = os.path.join(SINGLE_DIR, 'valid', 'valid_split.csv')

BWCL_TEST_PATH = os.path.join(SINGLE_DIR, 'test', 'logreg_bad_word_count.csv')
CVDCNN_TEST_PATH = os.path.join(SINGLE_DIR, 'test', 'char_vdcnn.csv')
CLOGREG_TEST_PATH = os.path.join(SINGLE_DIR, 'test', 'logreg_count.csv')
GDPCNN_TEST_PATH = os.path.join(SINGLE_DIR, 'test', 'glove_dpcnn.csv')
GLSTM_TEST_PATH = os.path.join(SINGLE_DIR, 'test', 'glove_lstm.csv')
GSCNN_TEST_PATH = os.path.join(SINGLE_DIR, 'test', 'glove_scnn.csv')
TFIDIF_LOGREG_TEST_PATH = os.path.join(SINGLE_DIR, 'test', 'logreg_tfidf.csv')
WLSTM_TEST_PATH = os.path.join(SINGLE_DIR, 'test', 'word_lstm.csv')

SAMPLE_SUBMISSION_PATH = os.path.join(SINGLE_DIR, 'test', 'sample_submission.csv')
ENSEMBLE_SUBMISSION_PATH = os.path.join(SINGLE_DIR, 'submission.csv')

# Validation Set Level 1 Predictions

In [16]:
bad_word_count_logreg_valid = pd.read_csv(BWCL_VALID_PATH)
char_vdcnn_valid = pd.read_csv(CVDCNN_VALID_PATH)
count_logreg_valid = pd.read_csv(CLOGREG_VALID_PATH)
glove_dpcnn_valid = pd.read_csv(GDPCNN_VALID_PATH)
glove_lstm_valid = pd.read_csv(GLSTM_VALID_PATH)
glove_scnn_valid = pd.read_csv(GSCNN_VALID_PATH)
tfidf_logreg_valid = pd.read_csv(TFIDIF_LOGREG_VALID_PATH)
word_lstm_valid = pd.read_csv(WLSTM_VALID_PATH)

labels_valid = pd.read_csv(VALID_LABELS_PATH)

In [17]:
bad_word_count_logreg_valid.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,20bc45f5014c1eca,0.024389,0.000826,0.003479,0.000882,0.008507,0.001497
1,948580166b8b8745,0.141035,0.002835,0.031479,0.002683,0.040060,0.003382
2,2c8519885db5c0bb,0.027942,0.000945,0.004199,0.000780,0.010168,0.001742
3,ed64d530a34aa973,0.147965,0.002652,0.034833,0.001616,0.046933,0.004547
4,a0dd897eafbada2c,0.020086,0.000756,0.002878,0.000653,0.007601,0.001468


In [18]:
X_valid = np.hstack([bad_word_count_logreg_valid.drop('id',axis=1), 
                     char_vdcnn_valid.drop('id',axis=1), 
                     count_logreg_valid.drop('id',axis=1), 
                     glove_dpcnn_valid.drop('id',axis=1), 
                     glove_lstm_valid.drop('id',axis=1), 
                     glove_scnn_valid.drop('id',axis=1), 
                     tfidf_logreg_valid.drop('id',axis=1), 
                     word_lstm_valid.drop('id',axis=1)])

y_valid_multilabel = labels_valid[LABEL_COLUMNS].values

In [19]:
X_valid.shape

(15958, 48)

# Test Set Level 1 Predictions

In [20]:
bad_word_count_logreg_test = pd.read_csv(BWCL_TEST_PATH)
char_vdcnn_test = pd.read_csv(CVDCNN_TEST_PATH)
count_logreg_test = pd.read_csv(CLOGREG_TEST_PATH)
glove_dpcnn_test = pd.read_csv(GDPCNN_TEST_PATH)
glove_lstm_test = pd.read_csv(GLSTM_TEST_PATH)
glove_scnn_test = pd.read_csv(GSCNN_TEST_PATH)
tfidf_logreg_test = pd.read_csv(TFIDIF_LOGREG_TEST_PATH)
word_lstm_test = pd.read_csv(WLSTM_TEST_PATH)

In [21]:
X_test = np.hstack([bad_word_count_logreg_test.drop('id',axis=1), 
                    char_vdcnn_test.drop('id',axis=1), 
                    count_logreg_test.drop('id',axis=1), 
                    glove_dpcnn_test.drop('id',axis=1), 
                    glove_lstm_test.drop('id',axis=1), 
                    glove_scnn_test.drop('id',axis=1), 
                    tfidf_logreg_test.drop('id',axis=1), 
                    word_lstm_test.drop('id',axis=1)])

# Ensemble Training

In [22]:
estimators = []
for i in range(6):
    y_valid_one_label = y_valid_multilabel[:,i]
    estimator = LogisticRegression()
    estimator.fit(X_valid, y_valid_one_label)
    estimators.append(estimator)

# Ensemble Prediction

In [23]:
y_test_pred = []
for estimator in estimators:
    y_test_pred_one_label = estimator.predict_proba(X_test)
    y_test_pred.append(y_test_pred_one_label)
y_test_pred = np.stack(y_test_pred, axis=1)[...,1]

# Submission

In [24]:
submission = pd.read_csv(SAMPLE_SUBMISSION_PATH)
submission[LABEL_COLUMNS] = y_test_pred
submission.to_csv(ENSEMBLE_SUBMISSION_PATH)
submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.990990,0.251604,0.971802,0.030149,0.911664,0.132895
1,0000247867823ef7,0.017128,0.000670,0.002327,0.000835,0.002777,0.001151
2,00013b17ad220c46,0.014774,0.000658,0.002285,0.000834,0.002923,0.001205
3,00017563c3f7919a,0.009214,0.000640,0.001530,0.000852,0.002894,0.001028
4,00017695ad8997eb,0.011690,0.000590,0.002571,0.000832,0.003391,0.001357
